In [ ]:
import json
import pathlib
import socket

In [ ]:
import netifaces

# Manual Preparation

To enable pos to interact with cloudlab on your behave, please ensure that:

- you have running management server using the `pos-daemon`; mail `pos` at `net.in.tum.de` for details
- your cloudlab key file
- a file containing your cloudlab password

are available on the management host running this notebook.
Moreover, add your cloudlab username.
Update the following variables as needed:

In [ ]:
CLOUDLAB_USER_NAME = 'hstubbe'
CLOUDLAB_KEY_FILE = '/home/debian/cloudlab.pem'
CLOUDLAB_PASS_FILE = '/home/debian/cloudlab.pass'

In [ ]:
try:
    CLOUDLAB_KEY_FILE = pathlib.Path(CLOUDLAB_KEY_FILE).resolve(strict=True)
    CLOUDLAB_PASS_FILE = pathlib.Path(CLOUDLAB_PASS_FILE).resolve(strict=True)
except FileNotFoundError as exception:
    raise ValueError("Ensure all required files are present, as mentioned in the cells above") from exception

# Automatic Configuration

To deploy the pos experiment controller, some configuration is required.
The following cells will try to determine a suitable external and internal network interface.

In [ ]:
try:
    IFADDRS = {
        name: netifaces.ifaddresses(name)
        for name in netifaces.interfaces()
        if name != "lo"
    }
    INTERNAL = "br0" # dedicated bridge interface for internal traffic
    UPLINK = next(name for name, config in IFADDRS.items() if netifaces.AF_INET in config)
except StopIteration as exception:
    raise ValueError("Unable to select interfaces, check host configuration") from exception

In [ ]:
CONFIG = {"metadata":{
    "project_id": "cloudlab",
    "management_server": {
        "name": socket.gethostname(),
        # password-based SSH authentication is disabled; use a stronger password before enabling it
        "password": "pos",
        "mac": {
            "internal": IFADDRS[INTERNAL][netifaces.AF_LINK][0]['addr'],
            "uplink": IFADDRS[UPLINK][netifaces.AF_LINK][0]['addr'],
        },
        "ip4": {
            "floating": "localhost",
            # any /12 in 172.16.0.0 that does not interfere with other networks
            "internal": "172.16.0.1/12",
            "uplink": f"{IFADDRS[UPLINK][netifaces.AF_INET][0]['addr']}/21",
        },
        "ip6": {
            # any /64 that does not interfere with other networks
            "internal": "fd20:59bc:8a88::1/64",
            # cloudlab does not offer IPv6, but pos assumes presence of such new/advanced features
            "uplink": "fd4f:ee21:c810::1/64",
        }
    },
    "experiment_server": [],
    "geni": {
        "username": str(CLOUDLAB_USER_NAME),
        "key_file": str(CLOUDLAB_KEY_FILE),
        "pass_file": str(CLOUDLAB_PASS_FILE),
    }
}}

In [ ]:
DEPLOY_POSD_JSON = pathlib.Path.cwd().joinpath("deploy-posd.json")
with DEPLOY_POSD_JSON.open("tw", encoding="utf8") as deploy_posd_json:
    json.dump(CONFIG, deploy_posd_json, indent=4)
DEPLOY_POSD_YAML = f"{pathlib.Path.home()}/pos-deployment/deploy-posd.yaml"

# pos Experiment Controller Deployment

Following configuration, the pos experiment contoller is deployed via Ansible.
This step can take a while, as cloudlab puts it "Patience please".

In [ ]:
!ansible-playbook \
    -i "localhost," \
    -e "ansible_user=debian" \
    -e "ansible_connection=local" \
    -e "deploy_posd_json={DEPLOY_POSD_JSON}" \
    -e "@{DEPLOY_POSD_JSON}" \
    "{DEPLOY_POSD_YAML}"

# Validate Deployment

After a successfull deployment, the following invokation of pos should work:

In [ ]:
import poslib.api as pos
pos.nodes.list_all()  # note: no nodes configured, yet

# Configure Experiment Nodes
Given that the pos experiment controller is deployed successfully, proceed with instantiating GENI nodes and making them available to pos.

In [ ]:
import json
import pathlib
import uuid

In [ ]:
import geni
import geni.portal
import poslib.api as pos

In [ ]:
# A plain Debian bullseye image, created with mandelstamm
disk_image = "https://pos-x.pages.gitlab.lrz.de/emulab/image-files/debian-10-mandelstamm.xml"
disk_image += f"?uuid={str(uuid.uuid4())}"
# Experiment node hardware type
hardware_type = "c220g2"
# Duration of the GENI allocation
duration = 16 * 60 # minutes
# Name of the GENI slice
slice_name = "pos-experiment"

In [ ]:
portal_context = geni.portal.Context()
request_rspec = portal_context.makeRequestRSpec()

experiment1 = geni.rspec.pg.RawPC("experiment-1")
experiment1.disk_image = disk_image
experiment1.hardware_type = hardware_type
experiment1_ifs = experiment1.addInterface("eth1"), experiment1.addInterface("eth2")
request_rspec.addResource(experiment1)

experiment2 = geni.rspec.pg.RawPC("experiment-2")
experiment2.disk_image = disk_image
experiment2.hardware_type = hardware_type
experiment2_ifs = experiment2.addInterface("eth1"), experiment2.addInterface("eth2")
request_rspec.addResource(experiment2)

link0 = geni.rspec.pg.Link()
link0.addInterface(experiment1_ifs[0])
link0.addInterface(experiment2_ifs[0])
request_rspec.addResource(link0)

link1 = geni.rspec.pg.Link()
link1.addInterface(experiment1_ifs[1])
link1.addInterface(experiment2_ifs[1])
request_rspec.addResource(link1)

In [ ]:
pos_geni_instantiate = pos.geni.instantiate(
    request_rspec,
    duration,
    slice_name,
)

In [ ]:
with pathlib.Path("variables.json").open("tw", encoding="utf8") as variables:
    json.dump({
        "global": {
            "loadgen_ingress_dev": 2,
            "loadgen_egress_dev": 1,
            "loadgen_ingress_if": "enp6s0f1",
            "loadgen_egress_if": "enp6s0f0",
            "dut_ingress_if": "enp6s0f0",
            "dut_egress_if": "enp6s0f1",
        },
        "loop": {
            "pkt_rate": [500000, 1000000, "...", 14880000],
        },
    }, variables)